In [4]:
# Importation des modules utilisés
!pip install pandas
!pip install sqlite3


import sqlite3
import pandas 

ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


In [5]:
# Création de la connexion
conn = sqlite3.connect("ClassicModel.sqlite")

# Récupération du contenu de Customers avec une requête SQL
customer = pandas.read_sql_query("SELECT * FROM Customers;", conn)
print(customer)


     customerNumber                    customerName contactLastName  \
0               103               Atelier graphique         Schmitt   
1               112              Signal Gift Stores            King   
2               114      Australian Collectors, Co.        Ferguson   
3               119               La Rochelle Gifts         Labrune   
4               121              Baane Mini Imports      Bergulfsen   
..              ...                             ...             ...   
117             486    Motor Mint Distributors Inc.       Hernandez   
118             487        Signal Collectibles Ltd.          Taylor   
119             489  Double Decker Gift Stores, Ltd           Hardy   
120             495            Diecast Collectables          Franco   
121             496               Kelly's Gift Shop         Snowden   

    contactFirstName           phone                  addressLine1  \
0             Carine      40.32.2555                54, rue Royale   
1      

In [6]:
# Récupération du contenu de Orders avce une requête SQL
orders = pandas.read_sql_query("SELECT * FROM Orders;", conn)
print(orders)

     orderNumber          orderDate       requiredDate        shippedDate  \
0          10100   2003/1/6 0:00:00  2003/1/13 0:00:00  2003/1/10 0:00:00   
1          10101   2003/1/9 0:00:00  2003/1/18 0:00:00  2003/1/11 0:00:00   
2          10102  2003/1/10 0:00:00  2003/1/18 0:00:00  2003/1/14 0:00:00   
3          10103  2003/1/29 0:00:00   2003/2/7 0:00:00   2003/2/2 0:00:00   
4          10104  2003/1/31 0:00:00   2003/2/9 0:00:00   2003/2/1 0:00:00   
..           ...                ...                ...                ...   
321        10421  2005/5/29 0:00:00   2005/6/6 0:00:00               NULL   
322        10422  2005/5/30 0:00:00  2005/6/11 0:00:00               NULL   
323        10423  2005/5/30 0:00:00   2005/6/5 0:00:00               NULL   
324        10424  2005/5/31 0:00:00   2005/6/8 0:00:00               NULL   
325        10425  2005/5/31 0:00:00   2005/6/7 0:00:00               NULL   

         status                                           comments  \
0    

In [10]:
##Lister les clients n’ayant jamais effecuté une commande 

rq1 = pandas.read_sql_query(
        #Explication des étapes de la requete dans l'ordre:
        # Sélection des colonnes customerNumber et customerName de la table Customers
        # Jointure à gauche entre la table Customers et la table Orders sur customerNumber cela inclut tous les clients, même ceux n'ayant pas de commande
        # Filtrer pour ne garder que les clients dont l'orderNumber est NULL (c'est-à-dire, ceux n'ayant pas de commande)
    """
    SELECT c.customerNumber, c.customerName
    FROM Customers c 
    LEFT JOIN Orders o ON c.customerNumber = o.customerNumber 
    WHERE o.orderNumber is NULL;
    """,
    conn)

print(rq1)

    customerNumber                    customerName
0              125              Havel & Zbyszek Co
1              168          American Souvenirs Inc
2              169               Porto Imports Co.
3              206      Asian Shopping Network, Co
4              223                 Natürlich Autos
5              237                   ANG Resellers
6              247        Messner Shopping Network
7              273               Franken Gifts, Co
8              293               BG&E Collectables
9              303                Schuyler Imports
10             307                Der Hund Imports
11             335       Cramer Spezialitäten, Ltd
12             348           Asian Treasures, Inc.
13             356            SAR Distributors, Co
14             361                 Kommission Auto
15             369          Lisboa Souveniers, Inc
16             376           Precious Collectables
17             409  Stuttgart Collectable Exchange
18             443        Feuer

In [12]:
##Pour chaque employé, le nombre de clients, le nombre de commandes et le montant total de celles-ci ;
rq2 =  pandas.read_sql_query(
    #Explication des étapes de la requete dans l'ordre:
    # Sélection des colonnes employeeNumber, firstName et lastName de la table Employees
    # ainsi que des calculs agrégés pour chaque employé (nombre de clients, nombre de commandes, montant total des commandes)
    # Jointure entre Employees et Customers pour lier l'employé aux clients qu'il gère
    # Jointure entre Customers et Orders pour lier les clients aux commandes qu'ils ont passées
    # Jointure entre Orders et Payments pour lier les commandes aux paiements effectués
    # Regroupement par employé
    """
    SELECT 
    e.employeeNumber,
    e.firstName,
    e.lastName,
    COUNT(DISTINCT c.customerNumber) AS numberOfCustomers, 
    COUNT(DISTINCT o.orderNumber) AS numberOfOrders,
    SUM(p.amount) AS totalOrderAmount
    FROM Employees e
    LEFT JOIN Customers c ON e.employeeNumber = c.salesRepEmployeeNumber
    LEFT JOIN Orders o ON c.customerNumber = o.customerNumber
    LEFT JOIN Payments p ON o.customerNumber = p.customerNumber
    GROUP BY e.employeeNumber, e.firstName , e.lastName;
    """,
    conn)
    
print(rq2)

    employeeNumber firstName   lastName  numberOfCustomers  numberOfOrders  \
0             1002     Diane     Murphy                  0               0   
1             1056      Mary  Patterson                  0               0   
2             1076      Jeff   Firrelli                  0               0   
3             1088   William  Patterson                  0               0   
4             1102    Gerard     Bondur                  0               0   
5             1143   Anthony        Bow                  0               0   
6             1165    Leslie   Jennings                  6              34   
7             1166    Leslie   Thompson                  6              14   
8             1188     Julie   Firrelli                  6              14   
9             1216     Steve  Patterson                  6              18   
10            1286  Foon Yue      Tseng                  7              17   
11            1323    George     Vanauf                  8      

In [14]:
#Idem pour chaque bureau (nombre de clients, nombre de commandes et montant total), avec en plus le nombre de clients d’un pays différent, s’il y en a ;

rq3 =  pandas.read_sql_query(
    # Sélection des informations du bureau, ainsi que des calculs agrégés sur les clients et les commandes
    # Jointure entre Offices et Employees pour lier chaque bureau à ses employés
    # Jointure entre Employees et Customers pour lier chaque employé à ses clients
    # Jointure entre Customers et Orders pour lier les clients aux commandes qu'ils ont passées
    # Jointure entre Orders et Payments pour lier les commandes aux paiements effectués
    # Regroupement par bureau
    """
    SELECT 
        of.officeCode,
        of.city,
        of.country AS officeCountry,
        COUNT(DISTINCT c.customerNumber) AS numberOfCustomers, 
        COUNT(DISTINCT o.orderNumber) AS numberOfOrders,
        SUM(p.amount) AS totalOrderAmount,
        COUNT(DISTINCT CASE
              WHEN c.country != of.country THEN c.customerNumber
              ELSE NULL
        END) AS customersFromDifferentCountry
    FROM Offices of
        LEFT JOIN Employees e ON e.officeCode = of.officeCode
        LEFT JOIN Customers c ON e.employeeNumber = c.salesRepEmployeeNumber
        LEFT JOIN Orders o ON c.customerNumber = o.customerNumber
        LEFT JOIN Payments p ON o.customerNumber = p.customerNumber
        GROUP BY of.officeCode, of.city, of.country;
       
    """,
    conn)
print(rq3)

  officeCode           city officeCountry  numberOfCustomers  numberOfOrders  \
0        1.0  San Francisco           USA                 12              48   
1        2.0         Boston           USA                 12              32   
2        3.0            NYC           USA                 15              39   
3        4.0          Paris        France                 29             106   
4        5.0          Tokyo         Japan                  5              16   
5        6.0         Sydney     Australia                 10              38   
6        7.0         London            UK                 17              47   

   totalOrderAmount  customersFromDifferentCountry  
0       13617508.61                              0  
1        2581034.07                              0  
2        3506160.15                              3  
3       27539012.59                             17  
4        1815463.17                              3  
5        4471955.56                      

In [16]:
#Pour chaque produit, donner le nombre de commandes, la quantité totale commandée, et le nombre de clients différents ;
rq4 =  pandas.read_sql_query(
    # Sélection des informations sur les produits, ainsi que des calculs agrégés sur les commandes et les clients
    # Jointure entre Products et OrderDetails pour lier chaque produit aux détails des commandes
    # Jointure entre OrderDetails et Orders pour lier les détails de commande aux commandes elles-mêmes
    #Regroupement par produit pour obtenir les résultats agrégés pour chaque produit
    """
    SELECT 
        p.Productcode,
        p.ProductName,
        COUNT(od.OrderNumber) AS numbOfOrders,
        SUM(od.QuantityOrdered) AS TotalQuantity,
        COUNT(DISTINCT o.CustomerNumber) AS NumberofCustomer
    FROM Products p
        LEFT JOIN OrderDetails od ON p.ProductCode= od.ProductCode
        LEFT JOIN Orders o ON od.OrderNumber=o.OrderNumber
        GROUP BY p.ProductCode, p.Productname   
    """,
    conn)
print(rq4)

    productCode                            productName  numbOfOrders  \
0      S10_1678  1969 Harley Davidson Ultimate Chopper            28   
1      S10_1949               1952 Alpine Renault 1300            28   
2      S10_2016                  1996 Moto Guzzi 1100i            28   
3      S10_4698   2003 Harley-Davidson Eagle Drag Bike            28   
4      S10_4757                    1972 Alfa Romeo GTA            28   
..          ...                                    ...           ...   
105   S700_3505                            The Titanic            27   
106   S700_3962                         The Queen Mary            27   
107   S700_4002              American Airlines: MD-11S            28   
108    S72_1253                       Boeing X-32A JSF            28   
109    S72_3212                             Pont Yacht            27   

     TotalQuantity  NumberofCustomer  
0           1026.0                26  
1            961.0                27  
2            999.0

In [18]:
#Donner le nombre de commande pour chaque pays, ainsi que le montant total des commandes et le montant total payé : on veut conserver les clients n’ayant jamais commandé dans le résultat final
rq5 =  pandas.read_sql_query(
    # Sélection des informations sur le pays des clients, ainsi que des calculs agrégés sur les commandes et les paiements
    # Jointure gauche entre la table Customers et Orders pour relier chaque client à ses commandes
    # Jointure gauche entre Orders et OrderDetails pour obtenir les informations sur les articles commandés
    # Jointure gauche avec la table Payments pour obtenir les paiements effectués par chaque client
    # Regroupement par pays
    """
        SELECT 
            c.Country,
            COUNT ( DISTINCT o.OrderNumber) AS NumberofOrders,
            SUM(P.Amount) AS TotalPaidAmount, 
            SUM(od.PriceEach * od.QuantityOrdered) AS TotalPaid 
        FROM Customers c 
            LEFT JOIN Orders o ON c.CustomerNumber = o.CustomerNumber 
            LEFT JOIN OrderDetails od ON o.OrderNumber = od.OrderNumber 
            LEFT JOIN  Payments p  ON p.CustomerNumber = c.CustomerNumber
            GROUP BY c.Country;
    """,
    conn)
print(rq5)

         country  NumberofOrders  TotalPaidAmount    TotalPaid
0      Australia              19     2.482541e+07   2182269.38
1        Austria               7     4.090982e+06    606187.59
2        Belgium               7     1.931535e+06    283705.44
3         Canada               7     4.487022e+06    448157.12
4        Denmark               7     7.001114e+06    781357.50
5        Finland               9     1.009620e+07    988745.73
6         France              37     3.141444e+07   3160296.75
7        Germany               7     4.971661e+06    576293.44
8      Hong Kong               2     7.805498e+05     48784.36
9        Ireland               2     9.241029e+05    115512.86
10        Israel               0              NaN          NaN
11         Italy              10     1.324310e+07    945208.16
12         Japan               6     4.837611e+06    496898.36
13   Netherlands               0              NaN          NaN
14   New Zealand              15     1.710337e+07   173

In [20]:
#On veut la table de contigence du nombre de commande entre la ligne de produits et le pays du client ;

rq6 =  pandas.read_sql_query(
    # Explication des étapes de la requête dans l'ordre :
    #Sélection de la ligne de produit (productLine) de la table Products
    #et du pays du client (country) de la table Customers ainsi que du calcul agrégé du nombre de commandes (NombreCommandes)
    #Jointure entre la table Orders et la table OrderDetails pour lier les commandes aux produits commandés
    #Jointure entre la table OrderDetails et la table Products pour lier chaque détail de commande à son produit
    #Jointure entre la table Orders et la table Customers pour lier chaque commande à son client
    # Groupement des résultats par ligne de produit (productLine) et pays du client (country)
    #Tri des résultats par ligne de produit, puis par pays pour une présentation logique des données
    """
    SELECT 
    Products.productLine AS LigneProduit,
    Customers.country AS PaysClient,
    COUNT(DISTINCT Orders.orderNumber) AS NombreCommandes
    FROM 
        Orders
    JOIN 
        OrderDetails ON Orders.orderNumber = OrderDetails.orderNumber
    JOIN 
        Products ON OrderDetails.productCode = Products.productCode
    JOIN 
        Customers ON Orders.customerNumber = Customers.customerNumber
    GROUP BY 
        Products.productLine, Customers.country
    ORDER BY 
        Products.productLine, Customers.country;

    """,
    conn)
print(rq6)


     LigneProduit PaysClient  NombreCommandes
0    Classic Cars  Australia               12
1    Classic Cars    Austria                5
2    Classic Cars    Belgium                2
3    Classic Cars     Canada                6
4    Classic Cars    Denmark                5
..            ...        ...              ...
121  Vintage Cars  Singapore                4
122  Vintage Cars      Spain               22
123  Vintage Cars     Sweden                4
124  Vintage Cars         UK               10
125  Vintage Cars        USA               67

[126 rows x 3 columns]


In [22]:
#On veut la même table croisant la ligne de produits et le pays du client, mais avec le montant total payé dans chaque cellule ;

rq7=  pandas.read_sql_query(
    #Sélection de la ligne de produit (productLine) de la table Products
    #et du pays du client (country) de la table Customers ainsi que du calcul agrégé du montant total payé (MontantTotalPaye)
    #Jointure entre la table Orders et la table OrderDetails pour lier les commandes aux produits commandés
    #Jointure entre la table OrderDetails et la table Products pour lier chaque détail de commande à son produit
    #Jointure entre la table Orders et la table Customers pour lier chaque commande à son client
    #Jointure entre la table Orders et la table Payments pour lier les commandes aux paiements effectués
    #Groupement des résultats par ligne de produit (productLine) et pays du client (country)
    #Tri des résultats par ligne de produit, puis par pays pour une présentation logique des données
    """
    SELECT 
    Products.productLine AS LigneProduit,
    Customers.country AS PaysClient,
    SUM(Payments.amount) AS MontantTotalPaye
    FROM 
        Orders
    JOIN 
        OrderDetails ON Orders.orderNumber = OrderDetails.orderNumber
    JOIN 
        Products ON OrderDetails.productCode = Products.productCode
    JOIN 
        Customers ON Orders.customerNumber = Customers.customerNumber
    JOIN 
        Payments ON Orders.customerNumber = Payments.customerNumber
    GROUP BY 
        Products.productLine, Customers.country
    ORDER BY 
        Products.productLine, Customers.country;
 
    """,
    conn)
print(rq7)

     LigneProduit PaysClient  MontantTotalPaye
0    Classic Cars  Australia        7504795.97
1    Classic Cars    Austria        1884419.42
2    Classic Cars    Belgium         166880.87
3    Classic Cars     Canada         774924.01
4    Classic Cars    Denmark        3678313.22
..            ...        ...               ...
121  Vintage Cars  Singapore        1941227.28
122  Vintage Cars      Spain       39878490.66
123  Vintage Cars     Sweden        1377094.16
124  Vintage Cars         UK        4673162.78
125  Vintage Cars        USA       47090713.86

[126 rows x 3 columns]


In [24]:

#Donner les 10 produits pour lesquels la marge moyenne est la plus importante (cf buyPrice et priceEach) ;

rq8=  pandas.read_sql_query(
    # Sélection du code produit (ProductCode) et du nom du produit (ProductName) de la table Products
    # ainsi que du calcul agrégé de la marge moyenne pour chaque produit (MargeMoyenne).
    # Jointure entre la table OrderDetails et la table Products pour lier chaque produit commandé à son produit respectif dans la table Products.
    # Groupement des résultats par nom de produit (ProductName) pour calculer la marge moyenne pour chaque produit.
    #Tri des résultats par marge moyenne (MargeMoyenne) de manière décroissante (DESC) pour afficher les produits avec la marge la plus élevée en premier.
    #Limitation des résultats aux 10 premiers produits (LIMIT 10).
    
    """
   SELECT
       p.ProductCode,
       p.ProductName,
       AVG(od.PriceEach - p.BuyPrice) AS MargeMoyenne 
       FROM OrderDetails od JOIN Products p ON od.ProductCode = p.ProductCode
       GROUP BY p.ProductName 
       ORDER BY MargeMoyenne DESC 
       LIMIT 10; 
        
    """,
    conn)
print(rq8)

  productCode                           productName  MargeMoyenne
0    S10_1949              1952 Alpine Renault 1300     99.006429
1    S10_4698  2003 Harley-Davidson Eagle Drag Bike     95.235000
2    S18_3232           1992 Ferrari 360 Spider red     83.334906
3    S12_2823                      2002 Suzuki XREO     83.201429
4    S18_2795                1928 Mercedes-Benz SSK     82.696786
5    S12_1108                     2001 Ferrari Enzo     81.043704
6    S12_3891                      1969 Ford Falcon     77.335926
7    S18_3685        1948 Porsche Type 356 Roadster     72.636800
8    S18_2870          1999 Indy 500 Monte Carlo SS     71.794400
9    S18_1749              1917 Grand Touring Sedan     70.432800


In [26]:

#Lister les produits (avec le nom et le code du client) qui ont été vendus à perte :
#   Si un produit a été dans cette situation plusieurs fois, il doit apparaître plusieurs fois,
#   Une vente à perte arrive quand le prix de vente est inférieur au prix d’achat ;

rq9=  pandas.read_sql_query(
    # Explication des étapes de la requête dans l'ordre :
    # Sélection du nom du produit (productName), du code du produit (productCode), du nom du client (customerName),
    #et du code du client (customerNumber), ainsi que du prix de vente (priceEach) et du prix d'achat (buyPrice)
    # Jointure entre la table OrderDetails et la table Orders sur le numéro de commande (orderNumber),afin d'associer chaque détail de commande à la commande correspondante.
    #Jointure entre la table Orders et la table Customers sur le numéro du client (customerNumber),afin d'associer chaque commande au client qui l'a passée.
    # Jointure entre la table OrderDetails et la table Products sur le code produit (productCode),afin d'associer chaque détail de commande au produit concerné.
    # Filtrage des résultats pour ne conserver que les lignes où le prix de vente est inférieur au prix d'achat(à perte),

    """
SELECT 
    Products.productName AS NomProduit,
    Products.productCode AS CodeProduit,
    Customers.customerName AS NomClient,
    Customers.customerNumber AS CodeClient,
    OrderDetails.priceEach AS PrixVente,
    Products.buyPrice AS PrixAchat
    FROM 
        OrderDetails
    JOIN 
        Orders ON OrderDetails.orderNumber = Orders.orderNumber
    JOIN 
        Customers ON Orders.customerNumber = Customers.customerNumber
    JOIN 
        Products ON OrderDetails.productCode = Products.productCode
    WHERE 
        OrderDetails.priceEach < Products.buyPrice;
    """,
    conn)

print(rq9)

                             NomProduit CodeProduit  \
0                1962 LanciaA Delta 16V    S10_4962   
1                    1934 Ford V8 Coupe    S18_2957   
2   18th Century Vintage Horse Carriage    S18_3136   
3                    1969 Corvair Monza    S12_3148   
4                1964 Mercedec Tour Bus    S18_2319   
..                                  ...         ...   
74               1962 LanciaA Delta 16V    S10_4962   
75                       1958 Setra Bus    S12_1666   
76          1913 Ford Model T Speedster    S18_2949   
77       1998 Chrysler Plymouth Prowler    S18_2238   
78                    2001 Ferrari Enzo    S12_1108   

                       NomClient  CodeClient  PrixVente  PrixAchat  
0   Online Diecast Creations Co.         363      61.99     103.42  
1   Online Diecast Creations Co.         363      29.87      34.35  
2   Online Diecast Creations Co.         363      47.04      60.74  
3                Vitachrome Inc.         181      54.33      89

In [28]:
#Lister les clients pour lesquels le montant total payé est supérieur aux montants totals des achats 
rq10=  pandas.read_sql_query(
    # Explication des étapes de la requête dans l'ordre :
    # Sélection du nom du client (customerName), du code du client (customerNumber), du montant total payé
    # Jointure entre la table Customers et la table Payments sur le numéro du client (customerNumber),afin de lier chaque paiement au client correspondant.
    # Jointure entre la table Customers et la table Orders sur le numéro du client (customerNumber),pour lier chaque commande au client qui l'a passée.
    # Jointure entre la table Orders et la table OrderDetails sur le numéro de commande (orderNumber), pour obtenir les détails de chaque commande, y compris le prix de vente (priceEach) et la quantité commandée (quantityOrdered).
    # Groupement des résultats par numéro et nom du client pour calculer les sommes des paiements et des achats par client.
    # Utilisation de la clause HAVING pour filtrer les résultats et ne conserver que les clients pour lesquels le montant total payé est supérieur au montant total des achats.
    """
    SELECT 
    Customers.customerName AS NomClient,
    Customers.customerNumber AS CodeClient,
    SUM(Payments.amount) AS MontantTotalPaye,
    SUM(OrderDetails.priceEach * OrderDetails.quantityOrdered) AS MontantTotalAchats
    FROM 
        Customers
    JOIN 
        Payments ON Customers.customerNumber = Payments.customerNumber
    JOIN 
        Orders ON Customers.customerNumber = Orders.customerNumber
    JOIN 
        OrderDetails ON Orders.orderNumber = OrderDetails.orderNumber
    GROUP BY 
        Customers.customerNumber, Customers.customerName
    HAVING 
        SUM(Payments.amount) > SUM(OrderDetails.priceEach * OrderDetails.quantityOrdered);
    """,
    conn)
print(rq10)

                         NomClient  CodeClient  MontantTotalPaye  \
0                Atelier graphique         103         169259.72   
1               Signal Gift Stores         112        2399781.32   
2       Australian Collectors, Co.         114       10745060.15   
3                La Rochelle Gifts         119        7226033.25   
4               Baane Mini Imports         121        3731174.08   
..                             ...         ...               ...   
93    Motor Mint Distributors Inc.         486        1924689.68   
94        Signal Collectibles Ltd.         487         753277.65   
95  Double Decker Gift Stores, Ltd         489         432228.48   
96            Diecast Collectables         495        1275476.04   
97               Kelly's Gift Shop         496        7540854.72   

    MontantTotalAchats  
0             72539.88  
1            248253.24  
2            803981.64  
3            540374.82  
4            466396.76  
..                 ...  
93      